In [1]:
from veering import taut_polytope
from veering import taut
from veering import taut_homology
from veering import taut_carried
from veering import fundamental_domain
from veering import taut_polynomial
from veering import transverse_taut
from sage.plot.contour_plot import ContourPlot
from itertools import combinations
from itertools import permutations
from veering import carried_surface

In [46]:
sig = 'fLLQcbecdeepuwsua_20102'
tri, angle = taut.isosig_to_tri_angle(sig)
rays = taut_polytope.taut_rays(sig) # rays (specified by surfaces which are described by solutions to branch equations) spanning the cone of carried classes
P = taut_polytope.projection_to_homology(sig) # multiply a surface (expressed as a solution to the branch equations) by this matrix to get its image in H_2(M, dM).

ray = rays[4]
ray

(0, 1, 2, 0, 3, 0, 1, 0, 0, 1)

In [47]:
carried_surface.genus_punctures(tri, angle, ray, return_surface = True)

(2, 2, <regina.engine.Triangulation2 object at 0x16eac6a70>)

In [13]:
taut_carried.boundary_cycles_from_surface(tri, angle, ray)

[[1, 0, 0, -1, -1, 0, 0, 0, 2, 3], [-1, 0, 0, 1, 1, 0, 0, 0, -2, -3]]

In [23]:
def get_directed_dual_graph(sig):
    tri, angle = taut.isosig_to_tri_angle(sig)
    G = DiGraph(loops = True, multiedges = True)
    top_bottom = transverse_taut.top_bottom_embeddings_of_faces(tri, angle)
    for face in tri.triangles():
        if face.embedding(0) in top_bottom[1]:
            G.add_edge(int(str(face.embedding(1))[0]), int(str(face.embedding(0))[0]), face.index())
        else:
            G.add_edge(int(str(face.embedding(0))[0]), int(str(face.embedding(1))[0]), face.index())
    return G

def get_edge_in_graph_from_face_label(graph, label):
    for edge in graph.edges():
        if edge[2] == label:
            return edge

def get_num_punctures(sig, surface):
    # This is another attempt at the above function.
    
    #surface = get_surf_from_point_in_homology(sig, point)
    G = get_directed_dual_graph(sig) # load the dual graph
    tri, angle = taut.isosig_to_tri_angle(sig) 
    boundaries = taut_carried.boundary_cycles_from_surface(tri, angle, surface) # load the list of faces which in the dual graph are homotopic to the boundary components. Notice the last input is just an example surface to work with.
    #print('faces traversed in the boundaries are:', boundaries)
    temp_boundaries = boundaries[:] # temporary list that we will manipulate to keep count
    count = 0 # This will keep count of the number of boundary components
    already_counted = []
    for l in range(len(temp_boundaries)): # loop over all the boundary components of the 3-manifold
        #print('already_counted:', already_counted)
        temp_boundary = temp_boundaries[l]
        temp_boundary = [abs(ele) for ele in temp_boundary] # since we don't care about direction and we want to subtract things later, just make everything positive.
        nonzero_indices = [] # we want to make a list of every face which actually occurs in a loop homotopic to a boundary component.
        for i in range(len(temp_boundary)):
            if temp_boundary[i] != 0:
                nonzero_indices.append(i)
        
        longest_possible_cycle = len(nonzero_indices) # the longest possible cycle is the number of faces which occur, since faces can't occur twice in a loop.

        while temp_boundary != [0]*len(temp_boundary): # while there are still face entries that haven't been accounted for in loops
            #print(count) # for testing
            for i in range(1, longest_possible_cycle + 1):
                #print('count:', count)
                #print('already counted:', already_counted)
                subset = list(combinations(nonzero_indices, i)) # make lists of all possible loops consisting of i faces
                #print('length', i, 'subsets of nonzero edges:', subset) # for testing
                for j in range(len(subset)): # for each potential i element loop
                    edges = []
                    for face in subset[j]:
                        edges.append(get_edge_in_graph_from_face_label(G,face)) 
                    #print('edges corresponding to', subset[j], 'are', edges) # for testing
                    subgraph = DiGraph(edges) # build the graph with the edges in this subset
                    to_check = [edge[2] for edge in edges]
                    #print('to check:', to_check)
                    to_checks = list(map(list,permutations(to_check)))
                    #print('to_checks:', to_checks)
                    #print('is loop:', subgraph.is_cycle(directed_cycle = False)) # for testing
                    if subgraph.is_cycle(directed_cycle = False): # if that subgraph forms a loop
                        if 0 not in [temp_boundary[face] for face in subset[j]]: # and temp_boundary still has nonzero entries for all the faces in that loop (so they haven't all been accounted for)
                            sub = min([temp_boundary[l] for l in subset[j]]) # subtract however many times this loops occurs from each entry in temp_boundary
                            for face in subset[j]:
                                temp_boundary[face] -= sub
                            #print(temp_boundary) # This is for testing
                            if [loop for loop in already_counted if loop in to_checks] == []: # if we haven't alreayd counted this loop
                                already_counted.append(to_check)
                                #count += sub # This wasn't aligning with the examples from the veering code function genus_punctures.
                                count += 1 # and add another boundary component to temp.
                    
    return count

In [24]:
get_num_punctures(sig, ray)

2

In [27]:
test = 'gvLQQcdeffeffffaafa_201102'
tri2, angle2 = taut.isosig_to_tri_angle(test)
tri2.vertices()

In [48]:
rays2 = taut_polytope.taut_rays(test)
ray2 = rays2[0]
taut_carried.boundary_cycles_from_surface(tri2, angle2, ray2)

[[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1],
 [-1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, -1, 0, -1, 0, 0, 0, -1, 0]]

In [45]:
a,b = var('a,b')
get_num_punctures(sig, (-a, 0, -a, -2*a - b, -4*a - 2*b, -a, -a, 0, -2*a - b, 0))

KeyboardInterrupt: 